In [126]:
%reload_ext autoreload
%autoreload 2

In [127]:

import sys
# just in case reload_ext does not work
# Remove the modules if they exist in sys.modules
for mod in ["typst_importer.typst_to_svg", "typst_importer.curve_utils", "typst_importer.svg_preprocessing"]:
    if mod in sys.modules:
        print(f"Removing {mod} from sys.modules")
        del sys.modules[mod]

# Now Python 'forgets' those imports. Reimport them:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

Removing typst_importer.typst_to_svg from sys.modules
Removing typst_importer.curve_utils from sys.modules
Removing typst_importer.svg_preprocessing from sys.modules


In [134]:
import bpy
import sys
import tempfile
import typst
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))

from typst_importer.notebook_utils import display_svg

#uv pip install lxml
#uv pip install numpy==1.26.4
# so that code highlighting works


temp_dir = Path(tempfile.gettempdir())
typst_file = temp_dir / "step1.typ"
svg_file = temp_dir / "step1.svg"

# Select Typst File

In [148]:
file_content = """
#set page(width: 550pt, height: auto, margin: 1cm, fill: none)

#import "@preview/codelst:2.0.2": sourcecode
#import "@preview/tablex:0.0.9": tablex
#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 20pt,
  stroke: 2pt + luma(0),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
for i in range(0,10):
  print(i)
```]
"""


typst_file.write_text(file_content)

353

In [149]:
import typst
print(typst.__version__)

0.14.0


In [151]:

typst.compile(str(typst_file), output=str(svg_file))

In [152]:
# typst_path = Path.home() / "projects/blender_typst_importer/docs/"
# typst_file = typst_path / "matrix.txt"
# typst_file = typst_path / "code.typ"
#typst_file = typst_path / "color_eq.txt"

# Manual SVG conversion + add to scene

In [153]:
from typst_importer.svg_preprocessing import preprocess_svg, stroke_to_filled_path

typst.compile(str(typst_file), output=str(svg_file))

step1_content = svg_file.read_text()
step2_content = preprocess_svg(step1_content)
step3_content = stroke_to_filled_path(step2_content)


display_svg(step1_content , width='500px')
display_svg(step3_content , width='500px')

FileNotFoundError: No such file or directory (os error 2)

[I 2025-10-29 16:17:22.971 ServerApp] 302 GET / (@127.0.0.1) 0.30ms
[I 2025-10-29 16:17:22.972 ServerApp] 302 GET / (@127.0.0.1) 0.22ms
[I 2025-10-29 16:17:22.980 ServerApp] 302 GET / (@127.0.0.1) 0.25ms
[I 2025-10-29 16:17:22.982 ServerApp] 302 GET / (@127.0.0.1) 0.31ms
[I 2025-10-29 16:17:22.982 ServerApp] 302 GET / (@127.0.0.1) 0.36ms
[I 2025-10-29 16:17:22.983 ServerApp] 302 GET / (@127.0.0.1) 0.43ms
[I 2025-10-29 16:17:22.983 ServerApp] 302 GET / (@127.0.0.1) 0.60ms
[I 2025-10-29 16:17:22.983 ServerApp] 302 GET / (@127.0.0.1) 0.64ms
[I 2025-10-29 16:17:22.984 ServerApp] 302 GET / (@127.0.0.1) 0.80ms
[I 2025-10-29 16:17:22.984 ServerApp] 302 GET / (@127.0.0.1) 0.91ms
[I 2025-10-29 16:17:22.986 LabApp] 302 GET /lab? (@127.0.0.1) 0.58ms
[I 2025-10-29 16:17:22.986 LabApp] 302 GET /lab? (@127.0.0.1) 0.70ms
[I 2025-10-29 16:17:22.987 LabApp] 302 GET /lab? (@127.0.0.1) 0.45ms
[I 2025-10-29 16:17:22.987 LabApp] 302 GET /lab? (@127.0.0.1) 0.55ms
[I 2025-10-29 16:17:22.992 LabApp] 302 GET /

# SVG Blender Operations

In [96]:
output_file = temp_dir / "step3.svg"
output_file.write_text(step3_content)

bpy.ops.import_curve.svg(filepath=str(output_file))
col = bpy.context.scene.collection.children['step3.svg']
col.name = "Formula"

for obj in col.objects:
    obj.scale = (100, 100, 100) # this is the old method, #better 
    # obj.data.transform(Matrix.Scale(scale_factor, 4))

NameError: name 'step3_content' is not defined

In [20]:
# Loop through each object in the collection and set its origin
bpy.ops.object.select_all(action='DESELECT')
if col.objects:
    # Set the first object as active
    bpy.context.view_layer.objects.active = col.objects[0]
    # Now we can safely set the mode to OBJECT
    bpy.ops.object.mode_set(mode='OBJECT') 
    for obj in col.objects:
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='MEDIAN')
        obj.select_set(False)

# Function overview


In [65]:
import tempfile
from pathlib import Path
from typst_importer.typst_to_svg import typst_to_blender_curves
typst_file =  Path(tempfile.gettempdir()) / "hello.typ"

header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a= b/c $"
typst_file.write_text(header+body)
collection = typst_to_blender_curves(typst_file)

In [64]:
from typst_importer.typst_to_svg import typst_express

c = typst_express("$ h+s+s d+d $", scale_factor=100, origin_to_char=False, convert_to_mesh=True)
print(c.processed_svg)

In [61]:
svg_content = """
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 41.7 5.3" width="41.7pt" height="5.3pt">
    <g>
        <g transform="translate(0 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(10 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#FFFFFF" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(20 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(30 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#FFFFFF" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
    </g>
</svg>
"""
from pathlib import Path
import tempfile
import bpy
from mathutils import Matrix


def create_material(color, name=""):
    """Create a new material with nodes setup for opacity."""
    mat = bpy.data.materials.new(name=name)
    mat.use_nodes = True
    nodes = mat.node_tree.nodes
    links = mat.node_tree.links

    nodes.clear()

    output = nodes.new("ShaderNodeOutputMaterial")
    output.location = (300, 0)
    principled = nodes.new("ShaderNodeBsdfPrincipled")
    principled.location = (0, 0)

    attr_node = nodes.new("ShaderNodeAttribute")
    attr_node.attribute_name = "opacity"
    attr_node.attribute_type = "OBJECT"
    attr_node.location = (-300, -100)

    principled.inputs["Base Color"].default_value = color

    links.new(principled.outputs["BSDF"], output.inputs["Surface"])
    links.new(attr_node.outputs["Fac"], principled.inputs["Alpha"])

    return mat


def deduplicate_materials(collection):
    """
    Deduplicate materials in a collection by reusing identical materials and giving them descriptive names.

    Args:
        collection: bpy.types.Collection - The collection containing objects whose materials need deduplication
    """
    materials_dict = {}

    for obj in collection.objects:
        if not obj.data.materials:
            continue

        current_mat = obj.data.materials[0]
        mat_key = tuple(current_mat.diffuse_color)

        if mat_key in materials_dict:
            obj.data.materials.clear()
            obj.data.materials.append(materials_dict[mat_key])
        else:
            rgb = current_mat.diffuse_color[:3]
            hex_color = "".join(f"{int(c*255):02x}" for c in rgb)
            new_mat = create_material(
                current_mat.diffuse_color, f"Mat{len(materials_dict)}_#{hex_color}"
            )
            materials_dict[mat_key] = new_mat

            obj.data.materials.clear()
            obj.data.materials.append(new_mat)

            if current_mat.users == 0:
                bpy.data.materials.remove(current_mat)


def cleanup_scene():
    """Clean up the Blender scene by removing all objects and collections."""
    bpy.ops.object.select_all(action="SELECT")
    bpy.ops.object.delete()

    for collection in bpy.data.collections:
        bpy.data.collections.remove(collection)

    bpy.ops.outliner.orphans_purge()


def setup_object(obj, scale_factor=200):
    """Setup individual object properties."""
    obj.data.transform(Matrix.Scale(scale_factor, 4))
    obj["opacity"] = 1.0
    obj.id_properties_ui("opacity").update(min=0.0, max=1.0, step=0.1)


def convert_to_mesh(obj):
    """Convert curve object to mesh while preserving properties."""
    if obj.type != "CURVE":
        return

    curve_data = obj.data
    original_name = obj.name.replace("Curve", "")

    bpy.context.view_layer.objects.active = obj
    obj.select_set(True)

    bpy.ops.object.convert(target="MESH")

    new_name = f"Mesh{original_name}"
    obj.name = new_name
    obj.data.name = new_name

    obj.select_set(False)
    bpy.data.curves.remove(curve_data)


# Main execution
temp_dir = Path(tempfile.gettempdir())
svg_file = temp_dir / "my_example.svg"
svg_file.write_text(svg_content)

cleanup_scene()

bpy.ops.import_curve.svg(filepath=str(svg_file))
imported_collection = bpy.context.scene.collection.children.get(svg_file.name)
imported_collection.name = "Hello"

for obj in imported_collection.objects:
    setup_object(obj)

deduplicate_materials(imported_collection)
bpy.ops.outliner.orphans_purge()

for obj in imported_collection.objects:
    convert_to_mesh(obj)